# 正規分布の条件付き分布

In [1]:
from shiori.utils.viewer import set_plot_params, sns, plt
from shiori.utils.path import ProjectPath
from pathlib import Path
import matplotlib.animation as animation
import numpy as np
set_plot_params()
name = Path().cwd().name + "/normal_cpd"
proj = ProjectPath(save_dst=name)
save_dir = proj.save_dir
save_dir.mkdir(parents=True, exist_ok=True)
plt.rcParams.update({
    "text.usetex": True,               # LaTeXを使う
    "font.family": "Arial",            # セリフ体
    "font.serif": ["Computer Modern"], # LaTeX標準フォント
})


In [9]:
# ==== パラメータ ====
mu_X, mu_Y = 1.0, 2.0
sigma_X, sigma_Y = 1.0, 1.5
rho = 0.7
cov = [[sigma_X**2, rho*sigma_X*sigma_Y],
       [rho*sigma_X*sigma_Y, sigma_Y**2]]
mean = [mu_X, mu_Y]

# ==== データ生成 ====
data = np.random.multivariate_normal(mean, cov, size=2000)

# ==== figure 準備 ====
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

# 左: 同時分布の散布図
sc = ax1.scatter(data[:, 0], data[:, 1], s=5, alpha=0.3, c="blue")
ax1.set_xlabel("X")
ax1.set_ylabel("Y")
ax1.set_title("Joint Distribution")

# 左に legend
legend_joint = r"$ (X,Y) \sim \mathcal{N}(\mu, \Sigma)$"
ax1.legend([sc], [legend_joint], loc="upper right", fontsize=10)

# 右: 条件付き分布 (Y|X=x)
y_vals = np.linspace(mu_Y - 4*sigma_Y, mu_Y + 4*sigma_Y, 300)
line_pdf, = ax2.plot([], [], color="red")
line_v = ax1.axvline(mu_X, color="red", linestyle="--")
title_pdf = ax2.set_title("")
legend_cond = ax2.legend(loc="upper right", fontsize=10)
ax2.set_xlim(y_vals.min(), y_vals.max())
ax2.set_ylim(0, 0.5)
ax2.set_xlabel("Y")
ax2.set_ylabel("Density")

# ==== アニメーション用のX値 ====
x_vals = np.linspace(mu_X - 2*sigma_X, mu_X + 2*sigma_X, 30)

def init():
    line_pdf.set_data([], [])
    line_v.set_xdata([mu_X, mu_X])
    title_pdf.set_text("")
    return line_pdf, line_v, title_pdf, legend_cond

def update(i):
    x = x_vals[i]
    cond_mean = mu_Y + rho * (sigma_Y/sigma_X) * (x - mu_X)
    cond_var = sigma_Y**2 * (1 - rho**2)
    cond_std = np.sqrt(cond_var)
    pdf = (1/(cond_std*np.sqrt(2*np.pi))) * np.exp(-(y_vals-cond_mean)**2/(2*cond_var))
    
    line_pdf.set_data(y_vals, pdf)
    line_v.set_xdata([x, x])
    title_pdf.set_text(rf"$Y|X={x:.2f}$")

    # legend 更新
    legend_label = rf"$Y|X \sim \mathcal{{N}}({cond_mean:.2f}, {cond_var:.2f})$"
    legend_cond = ax2.legend([line_pdf], [legend_label], loc="upper right", fontsize=10)

    return line_pdf, line_v, title_pdf, legend_cond

# ==== アニメーション作成 ====
ani = animation.FuncAnimation(
    fig, update, frames=len(x_vals), init_func=init,
    blit=True, interval=200, repeat=True
)

# ==== 保存 ====
ani.save(
    save_dir.joinpath("cpd.gif"),
    writer="pillow",
    dpi=200,   # 解像度
    fps=10
)

plt.close()
print("✅ GIF saved:", save_dir.joinpath("cpd.gif"))


/tmp/ipykernel_1443928/1337189250.py:30: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  legend_cond = ax2.legend(loc="upper right", fontsize=10)


✅ GIF saved: /home/takeda/project/shioru/src/.vuepress/public/assets/images/multivariate_distribution/normal_cpd/cpd.gif
